In [1]:
import pandas as pd
import plotly.express as px 
from urllib.request import urlopen
import json
import janitor
import plotly.io as pio
from json import loads, dumps
pio.renderers.default='notebook'

In [2]:
asthmaca = pd.read_csv('Sheets\\asthma_redownload_utf8.csv')
asthmaca = asthmaca.clean_names()
asthmaca

,county,year,strata,strata_name,age_group,number_of_hospitalizations,age_adjusted_hospitalization_rate,comment
0,California,2015,Total population,All ages,All ages,"27,262",7.0,NaN
1,Alameda,2015,Total population,All ages,All ages,"1,435",9.3,NaN
2,Alpine,2015,Total population,All ages,All ages,0,0.0,NaN
3,Amador,2015,Total population,All ages,All ages,28,7.5,NaN
4,Butte,2015,Total population,All ages,All ages,143,6.7,NaN
...,...,...,...,...,...,...,...,...
4479,Tulare,2020,Race/ethnicity,Multi-race,All ages,0,0.0,NaN
4480,Tuolumne,2020,Race/ethnicity,Multi-race,All ages,0,0.0,NaN
4481,Ventura,2020,Race/ethnicity,Multi-race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...
4482,Yolo,2020,Race/ethnicity,Multi-race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...


In [4]:
asthmaca = asthmaca.query('county != \'California\'')
asthmaca = asthmaca.query('year != 2015')
asthmaca = asthmaca.query('year != 2020')
asthmaca = asthmaca[asthmaca['strata'] == 'Race/ethnicity'].copy()
asthmaca

,county,year,strata,strata_name,age_group,number_of_hospitalizations,age_adjusted_hospitalization_rate,comment
1122,Alameda,2016,Race/ethnicity,White,All ages,138,2.6,NaN
1123,Alpine,2016,Race/ethnicity,White,All ages,0,0.0,NaN
1124,Amador,2016,Race/ethnicity,White,All ages,NaN,NaN,Count and rate suppressed in accordance with d...
1125,Butte,2016,Race/ethnicity,White,All ages,58,4.4,NaN
1126,Calaveras,2016,Race/ethnicity,White,All ages,NaN,NaN,Count and rate suppressed in accordance with d...
...,...,...,...,...,...,...,...,...
3653,Tulare,2019,Race/ethnicity,Multi-Race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...
3654,Tuolumne,2019,Race/ethnicity,Multi-Race,All ages,0,0.0,NaN
3655,Ventura,2019,Race/ethnicity,Multi-Race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...
3656,Yolo,2019,Race/ethnicity,Multi-Race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...


In [5]:
with open('Sheets\\California_County_Boundaries.json') as jsonfile:
    geojson = json.load(jsonfile)
geodf = pd.json_normalize(geojson, record_path='features')
fipmap = geodf.set_index('properties.CountyName')['properties.County_FIPS_ID'].to_dict()
asthmaca['fip'] = asthmaca['county'].map(fipmap)
asthmaca['fip'] = asthmaca['fip'].astype(str)

In [ ]:
fig = px.choropleth(
    asthmaca.query('year == 2019'),
    geojson=geojson,
    featureidkey='properties.County_FIPS_ID',
    locations='fip',
    color='age_adjusted_hospitalization_rate',
    range_color=(0, 25),
    hover_data=['county'],
    animation_frame='strata_name',
    labels={
        'age_adjusted_hospitalization_rate':'Age Adjusted Hospitalization Rate (per 10,000)'
    },
    color_continuous_scale='YlGn'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [10]:
fig.write_html('StrataAnalysisGeo_CorrectWidth.html')